# SWB analysis pipeline part 1: 

- create master df with roi labels for bipolar re-ref channels 
- determine roi coverage for each subject + by BDI score


*Created: 04/07/2024* \
*Updated: 09/15/2024* 


In [1]:
import numpy as np
import pandas as pd
import mne
from glob import glob
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
from scipy.stats import zscore, linregress, ttest_ind, ttest_rel, ttest_1samp, pearsonr, spearmanr
from mne.preprocessing.bads import _find_outliers
import os 
import joblib
import re
import datetime
import scipy
import random
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor

import warnings
warnings.filterwarnings('ignore')




In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('/sc/arion/projects/guLab/Alie/SWB/ephys_analysis/LFPAnalysis/')

In [4]:
from LFPAnalysis import lfp_preprocess_utils, sync_utils, analysis_utils, nlx_utils, oscillation_utils, statistics_utils

In [5]:
# Specify root directory for un-archived data and results 
base_dir = '/sc/arion/projects/guLab/Alie/SWB/'
anat_dir = f'{base_dir}ephys_analysis/recon_labels/'
neural_dir = f'{base_dir}ephys_analysis/data/'
behav_dir = f'{base_dir}swb_behav_models/data/behavior_preprocessed/'

date = datetime.date.today().strftime('%m%d%Y')
print(date)

09152024


## Data Loading

In [6]:
# Subj list for CCN 2024
subj_ids = list(pd.read_excel(f'{base_dir}ephys_analysis/subj_info/SWB_subjects.xlsx', sheet_name='Usable_Subjects', usecols=[0]).PatientID)
n_subj = len(subj_ids)
subj_ids


['MS002',
 'MS003',
 'MS009',
 'MS011',
 'MS015',
 'MS016',
 'MS017',
 'MS019',
 'MS022',
 'MS024',
 'MS025',
 'MS026',
 'MS027',
 'MS028',
 'MS029',
 'MS030',
 'MS033',
 'MS035',
 'MS041',
 'MS043',
 'MS048',
 'MS050',
 'DA8',
 'DA023',
 'DA026',
 'DA037',
 'DA039']

In [7]:
anat_info_all_subj = {f'{subj_id}': pd.read_csv(f'{anat_dir}{subj_id}_labels.csv') for subj_id in subj_ids}


In [8]:
anat_info_all_subj

{'MS002':        label BN246label          x          y          z      mni_x  \
 0     LaCaS1    A32sg_L  -6.784557  49.555827   0.149337  -6.382462   
 1    LaCaS10      A9l_L  -8.380923  61.155821  43.676991  -8.220073   
 2    LaCaS11      A9l_L  -8.780014  62.355821  48.469026  -8.542609   
 3    LaCaS12    Unknown  -8.780014  63.555820  53.261062  -8.436667   
 4     LaCaS2    A32sg_L  -6.784557  50.755827   4.941372  -6.368174   
 ..       ...        ...        ...        ...        ...        ...   
 153   RmOIF5  A12/47o_R  33.922762  42.355831  -8.636061  34.666463   
 154   RmOIF6  A12/47l_R  37.913675  41.555831  -6.240044  38.716568   
 155   RmOIF7  A12/47l_R  41.904589  40.755832  -3.844026  42.794042   
 156   RmOIF8  A12/47l_R  46.294594  39.555832  -1.048672  47.308597   
 157   RmOIF9     A45r_R  50.285508  38.755833   1.746682  51.450368   
 
          mni_y      mni_z       gm  \
 0    37.158688  -3.130044     Gray   
 1    48.757099  44.868611     Gray   
 2    49

In [9]:
anat_info_all_subj.keys()

dict_keys(['MS002', 'MS003', 'MS009', 'MS011', 'MS015', 'MS016', 'MS017', 'MS019', 'MS022', 'MS024', 'MS025', 'MS026', 'MS027', 'MS028', 'MS029', 'MS030', 'MS033', 'MS035', 'MS041', 'MS043', 'MS048', 'MS050', 'DA8', 'DA023', 'DA026', 'DA037', 'DA039'])

In [10]:
# load bp reref ch names 
reref_ch_all_subj  = {}

for subj_id in subj_ids:
    # load reref data 
    bp_data = mne.io.read_raw_fif(f'{neural_dir}{subj_id}/bp_ref_ieeg.fif', preload=False,verbose=12)
    # drop bad channels  
    bp_data = bp_data.drop_channels(bp_data.info['bads'])
    # save good re-ref ch names
    subj_ch = bp_data.ch_names
    # save info 
    reref_ch_all_subj[subj_id]  = subj_ch
    del bp_data
    
reref_ch_all_subj

Opening raw data file /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS002/bp_ref_ieeg.fif...
    Range : 0 ... 1083499 =      0.000 ...  2166.998 secs
Ready.
Opening raw data file /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS003/bp_ref_ieeg.fif...
    Range : 0 ... 1250999 =      0.000 ...  2501.998 secs
Ready.
Opening raw data file /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS009/bp_ref_ieeg.fif...
    Range : 0 ... 1077655 =      0.000 ...  2155.310 secs
Ready.
Opening raw data file /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS011/bp_ref_ieeg.fif...
    Range : 0 ... 1578124 =      0.000 ...  3156.248 secs
Ready.
Opening raw data file /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS015/bp_ref_ieeg.fif...
    Range : 0 ... 1108437 =      0.000 ...  2216.874 secs
Ready.
Opening raw data file /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS016/bp_ref_ieeg.fif...
    Range : 0 ... 1147968 =      0.000 ...  2295.936 secs
Ready.
Open

{'MS002': ['lacas1-lacas2',
  'lacas2-lacas3',
  'lacas3-lacas4',
  'lacas4-lacas5',
  'lacas5-lacas6',
  'lacas6-lacas7',
  'lacas7-lacas8',
  'lacas8-lacas9',
  'lacas9-lacas10',
  'lagit1-lagit2',
  'lagit2-lagit3',
  'lagit3-lagit4',
  'lagit6-lagit7',
  'lagit7-lagit8',
  'lagit8-lagit9',
  'laims1-laims2',
  'laims2-laims3',
  'laims3-laims4',
  'laims4-laims5',
  'laims5-laims6',
  'laims6-laims7',
  'laims10-laims11',
  'laims11-laims12',
  'laims12-laims13',
  'laims13-laims14',
  'lhplt1-lhplt2',
  'lhplt2-lhplt3',
  'lhplt3-lhplt4',
  'lhplt4-lhplt5',
  'lhplt8-lhplt9',
  'lhplt9-lhplt10',
  'lhplt10-lhplt11',
  'lhplt11-lhplt12',
  'lloif1-lloif2',
  'lloif2-lloif3',
  'lloif3-lloif4',
  'lloif6-lloif7',
  'lloif7-lloif8',
  'lloif8-lloif9',
  'lmoif2-lmoif3',
  'lmoif8-lmoif9',
  'lmoif9-lmoif10',
  'lmoif10-lmoif11',
  'lmoif11-lmoif12',
  'lmoif12-lmoif13',
  'lpips1-lpips2',
  'lpips2-lpips3',
  'lpips3-lpips4',
  'lpips4-lpips5',
  'lpips9-lpips10',
  'lpips10-lpips11'

## BP Re-Ref ROI Labels
- Choose which bipolar re-ref channel (anode vs cathode) should be used for ROI label
- Compile yba/manual anatomical information from anat dfs
- Find custom ROI label for yba labels

#### FUTURE IMPLEMENTATION: 
- USE SALMAN'S FUNCTION TO FIND OTHER DEFAULTS FOR MISSING ELECS
- DEAL WITH LESION ELECS 

In [11]:
# Anatomical Localization Info - Lab YBA ROIs  

# ROI labels for YBA regions
roi_label_info = pd.read_excel(f'{base_dir}ephys_analysis/LFPAnalysis/LFPAnalysis/YBA_ROI_labelled.xlsx',
                               usecols=['Hemisphere','Lobe','Region','Custom','Long.name'])
roi_label_info = roi_label_info.apply(lambda x: x.str.lower())
roi_label_info = roi_label_info.apply(lambda x: x.str.strip())
roi_label_info

,Hemisphere,Lobe,Region,Custom,Long.name
0,left,temporal,temporal pole,temporal pole,left superior temporal pole a
1,left,temporal,temporal pole,temporal pole,left superior temporal pole b
2,left,temporal,temporal pole,temporal pole,left superior temporal pole b
3,left,temporal,temporal pole,temporal pole,left superior temporal pole c
4,left,temporal,temporal pole,temporal pole,left superior temporal pole d
...,...,...,...,...,...
685,right,insula,insula,pins,right anterior long insular gyrus d
686,right,insula,insula,pins,right posterior long insular gyrus a
687,right,insula,insula,pins,right posterior long insular gyrus b
688,right,insula,insula,pins,right posterior long insular gyrus c


In [12]:
roi_label_info.Custom.unique().tolist()

['temporal pole',
 'temporal',
 'sts',
 nan,
 'parietal',
 'motor',
 'dmpfc',
 'dlpfc',
 'vlpfc',
 'ofc',
 'vmpfc',
 'phg',
 'acc',
 'mcc',
 'pcc',
 'hpc',
 'amy',
 'ains',
 'pins']

In [13]:
# roi_reref_labels_all_subj = {}
roi_reref_labels_master = []
unknown_subj = {}

for subj_id in subj_ids:
    # get subj reref ch
    reref_ch_names = reref_ch_all_subj[subj_id]
    # get anat file 
    anat_df = anat_info_all_subj[subj_id]
    anat_df['label'] = anat_df['label'].str.lower()
    anat_df['final_loc'] = [row.ManualExamination.lower().strip() if pd.isna(row.ManualExamination) == False else row.YBA_1.lower().strip() for index, row in anat_df.iterrows()]
    if np.sum(anat_df.final_loc.unique().tolist().count('unknown'))!=0:
        unknown_subj[subj_id] = np.sum(anat_df.final_loc.unique().tolist().count('unknown'))
    
    # check which elec should be used for roi label 
    loc4roi      = []
    ch_label4roi = []
    ch_type4roi  = []

    for ch in reref_ch_names:
        anode = ch.split('-')[0]
        cathode = ch.split('-')[1]
        anode_loc = anat_df.final_loc[anat_df.label == anode].str.lower().tolist()
        cathode_loc = anat_df.final_loc[anat_df.label == cathode].str.lower().tolist()
        if (anode_loc[0] == 'wm') | (anode_loc[0] == 'unknown'): # if anode is in wm or labeled unknown check cathode
            if (cathode_loc[0] == 'wm') | (cathode_loc[0] == 'unknown'): # check if cathode also wm or unknown 
                if (anode_loc[0] == 'wm') & (cathode_loc[0] == 'unknown'): # if anode is wm and cathode is unknown, use anode label
                    loc4roi.append('wm') # roi location detail 
                    ch_label4roi.append(anode) # ch label used for roi 
                    ch_type4roi.append('anode') # whether anode vs cathode was used 
                else: # if cathode is wm and anode is wm or unknown, use cathode, if cathode is unknown and anode is unknown use cathode
                    loc4roi.append(cathode_loc[0]) # roi location detail 
                    ch_label4roi.append(cathode) # ch label used for roi 
                    ch_type4roi.append('cathode') # whether anode vs cathode was used 
            else: # if cathode is in gm use cathode
                loc4roi.append(cathode_loc[0]) # roi location detail 
                ch_label4roi.append(cathode) # ch label used for roi
                ch_type4roi.append('cathode') # whether anode vs cathode was used 
        else: # if anode is in gm use anode 
            loc4roi.append(anode_loc[0]) # roi location detail 
            ch_label4roi.append(anode) # ch label used for roi
            ch_type4roi.append('anode') # whether anode vs cathode was used 
    
            
#     roi_reref_labels_all_subj[subj_id] = {'subj_id':[subj_id]*len(reref_ch_names),
#                                           'reref_ch_names':reref_ch_names,
#                                           'loc4roi':loc4roi,
#                                           'ch_label4roi':ch_label4roi,
#                                          'ch_type4roi':ch_type4roi }

    # add mni coordinate info for plot recon roi script
    mni_x = [anat_df.mni_x[anat_df.label == ch].item() for ch in ch_label4roi]
    mni_y = [anat_df.mni_y[anat_df.label == ch].item() for ch in ch_label4roi]
    mni_z = [anat_df.mni_z[anat_df.label == ch].item() for ch in ch_label4roi]
    
    roi_reref_subj_df = pd.DataFrame({'subj_id':[subj_id]*len(reref_ch_names),
                                      'reref_ch_names':reref_ch_names,
                                      'ch_label4roi':ch_label4roi,
                                      'ch_type4roi':ch_type4roi,
                                      'loc4roi':loc4roi,
                                      'mni_x':mni_x,
                                      'mni_y':mni_y,
                                      'mni_z':mni_z})
    
    roi_reref_labels_master.append(roi_reref_subj_df)
  

 
roi_reref_labels_master_df = pd.concat(roi_reref_labels_master).reset_index(drop=True)
    
    

In [14]:
roi_reref_labels_master_df

,subj_id,reref_ch_names,ch_label4roi,ch_type4roi,loc4roi,mni_x,mni_y,mni_z
0,MS002,lacas1-lacas2,lacas1,anode,left cingulate gyrus d,-6.382462,37.158688,-3.130044
1,MS002,lacas2-lacas3,lacas2,anode,left cingulate gyrus e,-6.368174,38.606223,2.270621
2,MS002,lacas3-lacas4,lacas3,anode,left cingulate gyrus f,-6.390079,39.941566,7.640265
3,MS002,lacas4-lacas5,lacas4,anode,left cingulate gyrus f,-6.914519,41.546899,12.993427
4,MS002,lacas5-lacas6,lacas5,anode,left cingulate gyrus g,-6.933604,42.745784,18.267675
...,...,...,...,...,...,...,...,...
1875,DA039,rtp6-rtp7,rtp6,anode,right superior middle temporal pole d,40.532268,7.354525,-33.253436
1876,DA039,rtp7-rtp8,rtp7,anode,right superior middle temporal pole d,43.911226,7.341369,-31.915085
1877,DA039,rsgcc6-rsgcc7,rsgcc7,cathode,right anterior pars triangularis b,42.378987,31.277597,2.471854
1878,DA039,rsgcc7-rsgcc8,rsgcc7,anode,right anterior pars triangularis b,42.378987,31.277597,2.471854


In [15]:
unknown_subj

{'MS009': 1, 'MS015': 1, 'MS022': 1, 'MS024': 1}

In [16]:
# add hemisphere to df by taking first letter of channel used for localization
roi_reref_labels_master_df['hemi'] = pd.Series(roi_reref_labels_master_df.ch_label4roi.str[0])

In [17]:
roi_reref_labels_master_df['hemi'].unique().tolist()

['l', 'r']

In [18]:
# add hemisphere to df by taking first letter of channel used for localization
roi_reref_labels_master_df['hemi'] = pd.Series(roi_reref_labels_master_df.ch_label4roi.str[0])
roi_reref_labels_master_df



,subj_id,reref_ch_names,ch_label4roi,ch_type4roi,loc4roi,mni_x,mni_y,mni_z,hemi
0,MS002,lacas1-lacas2,lacas1,anode,left cingulate gyrus d,-6.382462,37.158688,-3.130044,l
1,MS002,lacas2-lacas3,lacas2,anode,left cingulate gyrus e,-6.368174,38.606223,2.270621,l
2,MS002,lacas3-lacas4,lacas3,anode,left cingulate gyrus f,-6.390079,39.941566,7.640265,l
3,MS002,lacas4-lacas5,lacas4,anode,left cingulate gyrus f,-6.914519,41.546899,12.993427,l
4,MS002,lacas5-lacas6,lacas5,anode,left cingulate gyrus g,-6.933604,42.745784,18.267675,l
...,...,...,...,...,...,...,...,...,...
1875,DA039,rtp6-rtp7,rtp6,anode,right superior middle temporal pole d,40.532268,7.354525,-33.253436,r
1876,DA039,rtp7-rtp8,rtp7,anode,right superior middle temporal pole d,43.911226,7.341369,-31.915085,r
1877,DA039,rsgcc6-rsgcc7,rsgcc7,cathode,right anterior pars triangularis b,42.378987,31.277597,2.471854,r
1878,DA039,rsgcc7-rsgcc8,rsgcc7,anode,right anterior pars triangularis b,42.378987,31.277597,2.471854,r


In [19]:
(roi_reref_labels_master_df.loc4roi.unique().tolist())

['left cingulate gyrus d',
 'left cingulate gyrus e',
 'left cingulate gyrus f',
 'left cingulate gyrus g',
 'left superior frontal gyrus 1 a',
 'left superior frontal gyrus 2 b',
 'left superior frontal gyrus 2 c',
 'left amygdala inferior',
 'left superior middle temporal gyrus a',
 'left middle short insular gyrus c',
 'left anterior short insular gyrus b',
 'left anterior short insular gyrus a',
 'left pars opercularis c',
 'left superior frontal gyrus 4 d',
 'left hippocampus pes',
 'left hippocampus body',
 'left superior middle temporal gyrus d',
 'left inferior middle temporal gyrus d',
 'left frontal orbital 4 d',
 'left middle pars triangularis a',
 'left posterior pars triangularis a',
 'left frontal orbital 3 b',
 'left anterior pars triangularis b',
 'left posterior long insular gyrus b',
 'left anterior long insular gyrus a',
 'left middle frontal gyrus 7 a',
 'left superior frontal gyrus 5 d',
 'left pars opercularis a',
 'left anterior motor b',
 'left posterior motor i

In [20]:
roi_label_info.Custom[~roi_label_info.Custom.isnull()].unique()

array(['temporal pole', 'temporal', 'sts', 'parietal', 'motor', 'dmpfc',
       'dlpfc', 'vlpfc', 'ofc', 'vmpfc', 'phg', 'acc', 'mcc', 'pcc',
       'hpc', 'amy', 'ains', 'pins'], dtype=object)

In [21]:
yba_roi = roi_label_info.Custom[~roi_label_info.Custom.isnull()].unique().tolist()
all_roi = yba_roi+['caudate','thalamus']
all_roi

['temporal pole',
 'temporal',
 'sts',
 'parietal',
 'motor',
 'dmpfc',
 'dlpfc',
 'vlpfc',
 'ofc',
 'vmpfc',
 'phg',
 'acc',
 'mcc',
 'pcc',
 'hpc',
 'amy',
 'ains',
 'pins',
 'caudate',
 'thalamus']

In [22]:
#### currently there are a few bad ch with locs listed as 'wm', 'gm', 'unknown' 
bad_rois = ['wm', 'gm', 'unknown','oob'] # no oob but adding just in case 
roi_labels = []

#### add roi to df 
for idx, row in roi_reref_labels_master_df.iterrows():
    loc = row.loc4roi
    if any(loc in x  for x in bad_rois): #pd.Series(row['loc4roi']).isin(bad_rois)[0]
        roi_labels.append(np.nan)
    else: 
        if roi_reref_labels_master_df.loc4roi.str.contains('thalamus').iloc[idx]: 
            roi = 'thalamus'
            roi_labels.append(roi)
        elif roi_reref_labels_master_df.loc4roi.str.contains('caudate').iloc[idx]: 
            roi = 'caudate'
            roi_labels.append(roi)
        elif pd.Series(row['loc4roi'].strip()).isin(roi_label_info['Long.name'])[0]:
            roi = roi_label_info['Custom'][np.where(row['loc4roi'].strip() == roi_label_info['Long.name'])[0]].item() 
            roi_labels.append(roi)
        else: 
            roi_labels.append(np.nan)
    
roi_reref_labels_master_df['roi'] = roi_labels


In [23]:
#### list of labels without ROIs - either poorly labeled or isn't labeled in our YBA roi key
bad_ch_df = roi_reref_labels_master_df[roi_reref_labels_master_df.roi.isnull()]
bad_ch_df.loc4roi.unique().tolist()

['left inferior middle temporal gyrus d',
 'right inferior middle temporal gyrus d',
 'left inferior middle temporal gyrus a',
 'right somatosensory m',
 'wm',
 'gm',
 'left supramarginal gyrus 4 a',
 'left somatosensory m',
 'left medial occipitotemporal gyrus d',
 'left inferior middle temporal gyrus e',
 'right basal inferior temporal gyrus b',
 'right inferior middle temporal gyrus f',
 'unknown',
 'left inferior middle temporal gyrus b',
 'left inferior middle temporal gyrus c',
 'left supramarginal gyrus 3 a',
 'left somatosensory f',
 'right medial occipitotemporal gyrus a',
 'left supramarginal gyrus 5 e',
 'left supramarginal gyrus 1 b',
 'left superior angular gyrus b',
 'right inferior middle temporal gyrus b',
 'right superior angular gyrus b',
 'left medial occipitotemporal gyrus e',
 'right inferior middle temporal gyrus a',
 'right medial occipitotemporal gyrus e',
 'left basal inferior temporal gyrus e',
 'left lateral inferior temporal gyrus f',
 'left inferior middle 

In [24]:
roi_reref_labels_master_df

,subj_id,reref_ch_names,ch_label4roi,ch_type4roi,loc4roi,mni_x,mni_y,mni_z,hemi,roi
0,MS002,lacas1-lacas2,lacas1,anode,left cingulate gyrus d,-6.382462,37.158688,-3.130044,l,acc
1,MS002,lacas2-lacas3,lacas2,anode,left cingulate gyrus e,-6.368174,38.606223,2.270621,l,acc
2,MS002,lacas3-lacas4,lacas3,anode,left cingulate gyrus f,-6.390079,39.941566,7.640265,l,acc
3,MS002,lacas4-lacas5,lacas4,anode,left cingulate gyrus f,-6.914519,41.546899,12.993427,l,acc
4,MS002,lacas5-lacas6,lacas5,anode,left cingulate gyrus g,-6.933604,42.745784,18.267675,l,acc
...,...,...,...,...,...,...,...,...,...,...
1875,DA039,rtp6-rtp7,rtp6,anode,right superior middle temporal pole d,40.532268,7.354525,-33.253436,r,temporal pole
1876,DA039,rtp7-rtp8,rtp7,anode,right superior middle temporal pole d,43.911226,7.341369,-31.915085,r,temporal pole
1877,DA039,rsgcc6-rsgcc7,rsgcc7,cathode,right anterior pars triangularis b,42.378987,31.277597,2.471854,r,vlpfc
1878,DA039,rsgcc7-rsgcc8,rsgcc7,anode,right anterior pars triangularis b,42.378987,31.277597,2.471854,r,vlpfc


In [25]:
roi_reref_labels_master_df.to_csv(f'{base_dir}ephys_analysis/results/roi_reref_labels_master.csv')
roi_reref_labels_master_df.to_csv(f'{base_dir}ephys_analysis/results/roi_info/roi_reref_labels_master.csv')



In [26]:
roi_reref_labels_master_df['roi'][roi_reref_labels_master_df.roi.isin(['dlpfc',
                                                                       'vlpfc','dmpfc','ofc','ains','pins', 'acc','hpc',
                                                                       'amy'])].value_counts()

roi
dmpfc    225
ofc      202
acc      193
hpc      135
amy      113
dlpfc    107
vlpfc     82
ains      71
pins      60
Name: count, dtype: int64

In [27]:
counts_all_subj = roi_reref_labels_master_df.groupby('subj_id')['roi'].value_counts()

roi_subj_counts = []

for subj_id in subj_ids:
    subj_roi_nums = counts_all_subj[subj_id]
    count_df = pd.DataFrame()
    count_df['subj_id'] = [subj_id]*len(subj_roi_nums.to_list())
    count_df['roi'] = subj_roi_nums.index.tolist()
    count_df['count'] = subj_roi_nums.to_list()
    roi_subj_counts.append(count_df)

roi_subj_counts = pd.concat(roi_subj_counts)

In [28]:
roi_subj_counts[roi_subj_counts.subj_id=='MS048']

,subj_id,roi,count
0,MS048,hpc,24
1,MS048,dmpfc,16
2,MS048,ofc,15
3,MS048,acc,11
4,MS048,dlpfc,7
5,MS048,temporal,7
6,MS048,thalamus,7
7,MS048,amy,6
8,MS048,ains,5
9,MS048,sts,3


In [29]:
roi_subj_counts.to_csv(f'{base_dir}ephys_analysis/results/roi_subj_counts_{date}.csv')
roi_subj_counts.to_csv(f'{base_dir}ephys_analysis/results/roi_info/roi_subj_counts.csv')


In [30]:
roi_reref_labels_master_df

,subj_id,reref_ch_names,ch_label4roi,ch_type4roi,loc4roi,mni_x,mni_y,mni_z,hemi,roi
0,MS002,lacas1-lacas2,lacas1,anode,left cingulate gyrus d,-6.382462,37.158688,-3.130044,l,acc
1,MS002,lacas2-lacas3,lacas2,anode,left cingulate gyrus e,-6.368174,38.606223,2.270621,l,acc
2,MS002,lacas3-lacas4,lacas3,anode,left cingulate gyrus f,-6.390079,39.941566,7.640265,l,acc
3,MS002,lacas4-lacas5,lacas4,anode,left cingulate gyrus f,-6.914519,41.546899,12.993427,l,acc
4,MS002,lacas5-lacas6,lacas5,anode,left cingulate gyrus g,-6.933604,42.745784,18.267675,l,acc
...,...,...,...,...,...,...,...,...,...,...
1875,DA039,rtp6-rtp7,rtp6,anode,right superior middle temporal pole d,40.532268,7.354525,-33.253436,r,temporal pole
1876,DA039,rtp7-rtp8,rtp7,anode,right superior middle temporal pole d,43.911226,7.341369,-31.915085,r,temporal pole
1877,DA039,rsgcc6-rsgcc7,rsgcc7,cathode,right anterior pars triangularis b,42.378987,31.277597,2.471854,r,vlpfc
1878,DA039,rsgcc7-rsgcc8,rsgcc7,anode,right anterior pars triangularis b,42.378987,31.277597,2.471854,r,vlpfc


# Check anat by BDI

In [31]:
bdi_list = pd.read_excel(f'{base_dir}ephys_analysis/subj_info/SWB_subjects.xlsx', sheet_name='Usable_Subjects', usecols=[3])
bdi_list = list(bdi_list.SWB_BDI)
# list(zip(subj_ids,bdi_list))
subj_info_df = pd.DataFrame({'subj_id':subj_ids,'bdi':bdi_list})
subj_info_df

,subj_id,bdi
0,MS002,14
1,MS003,8
2,MS009,16
3,MS011,13
4,MS015,26
5,MS016,10
6,MS017,26
7,MS019,12
8,MS022,10
9,MS024,16


In [32]:
high_bdi_subj = subj_info_df.subj_id[subj_info_df.bdi>=20]
low_bdi_subj = subj_info_df.subj_id[subj_info_df.bdi<20]


In [33]:
roi_counts_by_bdi = []

for region in roi_subj_counts.roi.unique().tolist():
    total_count  = roi_reref_labels_master_df['roi'][roi_reref_labels_master_df.roi == region].value_counts().values[0]
    high_bdi_num = np.sum(roi_subj_counts['count'][(roi_subj_counts.subj_id.isin(high_bdi_subj))
                                                   &(roi_subj_counts.roi==region)].values)
    low_bdi_num = np.sum(roi_subj_counts['count'][(roi_subj_counts.subj_id.isin(low_bdi_subj))
                                                  &(roi_subj_counts.roi==region)].values)
    region_bdi_counts = pd.DataFrame({'roi':region,'total':total_count,'high_bdi':high_bdi_num,'low_bdi':low_bdi_num},index=[0])
    roi_counts_by_bdi.append(region_bdi_counts)
    
roi_counts_by_bdi = pd.concat(roi_counts_by_bdi).reset_index(drop=True)

    
    

In [34]:
roi_counts_by_bdi[roi_counts_by_bdi.roi.isin(['dlpfc','vlpfc','dmpfc','ofc','ains','pins', 'acc','hpc',
                                                                       'amy'])]

,roi,total,high_bdi,low_bdi
0,vlpfc,82,21,61
1,dmpfc,225,54,171
3,ofc,202,60,142
4,acc,193,47,146
5,ains,71,18,53
6,dlpfc,107,23,84
7,hpc,135,28,107
8,amy,113,28,85
9,pins,60,21,39


In [35]:
roi_counts_by_bdi.to_csv(f'{base_dir}ephys_analysis/results/roi_counts_by_bdi_{date}.csv')
roi_counts_by_bdi.to_csv(f'{base_dir}ephys_analysis/results/roi_info/roi_counts_by_bdi.csv')
